In [1]:
!pip install transformers

import transformers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pymongo import MongoClient
import transformers
import torch
from torch.utils.data import Dataset, DataLoader
import sklearn

C:\Users\Igo\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class config:
#     MODEL = 'bert-base-uncased'
#     MODEL = 'ProsusAI/finbert'
    MODEL = 'roberta-base'

    HIDDEN = 768

    MAX_LENGTH = 64
    TRAIN_BATCH_SIZE = 32
    VALID_BATCH_SIZE = 32

    EPOCHS = 10

    LR = (2e-5, 3e-5, 5e-5)
    EPS = 1e-8

    SEED = 23

In [3]:
connection=MongoClient("mongodb://192.168.0.18:27017/crawler.contents")
db=connection.get_database()
data = pd.DataFrame(list(db.contents.find()))
data = data.drop(columns=['visited','created_at','contentType','date','icon'])
data = data.sample(frac=1).reset_index(drop=True)
data = data[(data['classes_summary'].str.len() > 0)].reset_index(drop=True)

In [4]:
print(data)
ohe = pd.concat([data,data['classes_summary'].fillna("").map(lambda x: ",".join(x)).str.get_dummies(sep=",")],axis=1)
to_remove = ['ARS','MXN','OIL','XAU','TRY','INDEX','BTC','TWD','RUB','CHF']
ohe = ohe.drop(labels=to_remove,axis=1)
classes = ohe.columns.difference(data.columns).tolist()
data = ohe

# data['sentiment'] = encoder.fit_transform((data['Sentiment']))
# data['emotion'] = encoder.fit_transform((data['Emotion']))


data['classes'] = list(data[classes].values)
# classes = data['classes']
labels = list(data.classes.values)
# print(data)


# data = data.sample(n=20000)

data['text'] = data['summary']

data = data[['text', 'classes']]

print(data)

                           _id  \
0     62444779de4f8e740ef17d01   
1     62444788de4f8e740ef17d37   
2     623d642a6e887c1f5ed1a680   
3     62495f2ac461dc595d8961d2   
4     623d631a6e887c1f5ed1a4e6   
...                        ...   
7797  62854d577257d3c8eabe3cfb   
7798  624433da0524426478c687a4   
7799  626689b139c405c10597fd2b   
7800  6249626ba23aaff290bfc687   
7801  6244407d4bbbb4df3b921617   

                                                  title  \
0                        Euro pauses after recent gains   
1        Investors eye NZ inflation, manufacturing data   
2     Dollar slides vs yen, euro as U.S. stocks, Tre...   
3     Euro Outlook Latest: EUR/USD Slides Ahead of M...   
4     Dollar gains vs yen, others after U.S. jobs re...   
...                                                 ...   
7797  Japanese Yen Technical Analysis: USD/JPY, EUR/...   
7798  European Stocks Drop as Yields Tick Higher, EU...   
7799  Japan’s finance ministry official denies repor...   
7

In [5]:
class TextClassificationDataset:

    def __init__(self,
                 texts,
                 labels):

        self.texts = texts
        self.labels = labels
        self.max_seq_length = config.MAX_LENGTH

        self.tokenizer = transformers.RobertaTokenizer.from_pretrained(
            config.MODEL
        )

        self.sep_vid = self.tokenizer.vocab["[SEP]"]
        self.cls_vid = self.tokenizer.vocab["[CLS]"]
        self.pad_vid = self.tokenizer.vocab["[PAD]"]

    def __len__(self):

        return len(self.texts)

    def __getitem__(self, item):

        texts = str(self.texts[item])
        texts = " ".join(texts.split())
        inputs = self.tokenizer.encode_plus(
            texts,
            None,
            add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            max_length=self.max_seq_length,
            return_token_type_ids=False,
            return_tensors="pt",
        )

        true_seq_length = len(inputs['input_ids'][0])
        pad_size = self.max_seq_length - true_seq_length
        pad_ids = torch.Tensor([self.pad_vid] * pad_size).long()
        ids = torch.cat((inputs['input_ids'][0], pad_ids))


        output_dict = {
            "ids": ids.flatten(),
            'attention_mask': inputs["attention_mask"][0].flatten(),
            'target' : torch.tensor(self.labels[item], dtype=torch.long)

        }

        

        return output_dict

In [6]:
SEED = 42

n_classes = len(classes)

train, val = train_test_split(
    data, test_size=0.30, random_state=SEED)


train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)

In [7]:
from transformers import BertForSequenceClassification, BertModel
import torch
import torch.nn as nn





class Model(nn.Module):
    def __init__(self, n_classes):
        super(Model, self).__init__()
        self.bert = BertModel.from_pretrained(config.MODEL)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(config.HIDDEN, n_classes)

            
    
    def forward(self, input_ids, attention_mask=None):
        pooled_output = self.bert(
          input_ids=input_ids,
          attention_mask=attention_mask
        ).pooler_output
        output = self.drop(pooled_output)
        # output = self.softmax(output)
        return self.out(output)

In [8]:
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup
import torch
from torch.utils.data import DataLoader
import numpy as np
import time
import datetime
import random
from tqdm import tqdm


traindata = TextClassificationDataset(
    train.text.to_numpy(), 
    train.classes.to_numpy()
    )

trainLoader = DataLoader(
    traindata, 
    shuffle=True, 
    batch_size=config.TRAIN_BATCH_SIZE)


valdata = TextClassificationDataset(
    val.text.to_numpy(), 
    train.classes.to_numpy()
    )

valLoader = DataLoader(
    valdata, 
    shuffle=True, 
    batch_size=config.TRAIN_BATCH_SIZE)

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 878k/878k [00:00<00:00, 1.26MB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 446k/446k [00:00<00:00, 749kB/s]
Downloading: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 481/481 [00:00<00:00, 120kB/s]


AttributeError: 'RobertaTokenizer' object has no attribute 'vocab'

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")

    print("Cuda available")

else:
    print("No GPU's available")
    device = torch.device("cpu")

In [ ]:
TOTAL_STEPS = len(trainLoader)*config.EPOCHS

model = Model(n_classes)
model.to(device)

optimizer = AdamW(
    model.parameters(),
    lr = config.LR[-1],
    eps=config.EPS
    )
scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=TOTAL_STEPS
)

def accuracy_check(y_pred, y_true):
    y_pred = [pl>0.5 for pl in y_pred]
    
    metrics = { 
                'exact': sklearn.metrics.accuracy_score(y_true, y_pred, normalize=True, sample_weight=None),
                'hamming':sklearn.metrics.hamming_loss(y_true, y_pred),
                'recall': sklearn.metrics.recall_score(y_true=y_true, y_pred=y_pred, average='samples'),
                'precision': sklearn.metrics.precision_score(y_true=y_true, y_pred=y_pred, average='samples'),
                'f1': sklearn.metrics.f1_score(y_true=y_true, y_pred=y_pred, average='samples')
        
                                       }

    temp = 0
    for i in range(y_true.shape[0]):
        temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
    return temp / y_true.shape[0], metrics

def format_time(elasped):
    elasped_rounded = int(round(elasped))

    return str(datetime.timedelta(seconds=elasped_rounded))

random.seed(config.SEED)
np.random.seed(config.SEED)
torch.manual_seed(config.SEED)
torch.cuda.manual_seed(config.SEED)

training_stats = []

total_t0 = time.time()



def loss_fn(outputs, targets):
    return nn.CrossEntropyLoss()(outputs, targets.type(torch.FloatTensor).to(device))

for epoch in range(0, config.EPOCHS):
    print("")
    print("============EPOCHS {:}/{:}=============".format(epoch + 1, config.EPOCHS))
    print("Training")

    t0 = time.time()

    total_train_loss = 0

    model.train()
    for step, batch in enumerate(trainLoader):
       

        if step % 40 == 0 and not step == 0: 
            elapsed = format_time(time.time() - t0)


            print(" Batch {:>5,} of {:>5,}. Elasped: {:}".format(step, len(trainLoader), elapsed))

        b_input_ids = batch['ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['target'].to(device)
        
        model.zero_grad()

        output = model(
            b_input_ids,
            attention_mask = b_input_mask,
            )
        # print(output)
        # output = torch.nn.functional.softmax(output, dim=1)

        # print(output)

        # lossfn = torch.nn.CrossEntropyLoss().to(device)
        loss = loss_fn(output, b_labels).to(device)
        
        total_train_loss+=loss.item()



        loss.backward()


        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()


        avg_train_loss = total_train_loss/len(trainLoader)


        training_time = format_time(time.time() - t0)
        

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))

    #=========================================
    #           Validation
    #=========================================

    print("")
    print("Running Validation")
    
    t0 = time.time()

    model.eval()
    metrics = {'exact':0,'f1':0,'recall':0, 'hamming':0, 'precision':0}
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_step = 0

    for batch in valLoader:

        b_input_ids = batch['ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        b_labels = batch['target'].to(device)

        with torch.no_grad():
            model.zero_grad()

            output = model(
                b_input_ids,
                attention_mask = b_input_mask,
                )

        # output = torch.nn.functional.softmax(output)

        # lossfn = torch.nn.CrossEntropyLoss().to(device)
        loss = loss_fn(output, b_labels).to(device)
        
        total_eval_loss+=loss.item()

        output = output.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        acc, met = accuracy_check(output, label_ids)
        
        total_eval_accuracy += acc
        metrics['exact'] += met['exact']
        metrics['f1'] += met['f1']
        metrics['precision'] += met['precision']
        metrics['recall'] += met['recall']
        metrics['hamming'] += met['hamming']

        
    
    avg_val_accuracy = total_eval_accuracy/len(valLoader)

    print("   Accuracy: {0:.2f}".format(avg_val_accuracy))
    print("   exact: {0:.2f}".format(metrics['exact']/len(valLoader)))
    print("   f1: {0:.2f}".format(metrics['f1']/len(valLoader)))
    print("   precision: {0:.2f}".format(metrics['precision']/len(valLoader)))
    print("   recall: {0:.2f}".format(metrics['recall']/len(valLoader)))
    print("   hamming: {0:.2f}".format(metrics['hamming']/len(valLoader)))

    avg_val_loss = total_eval_loss/len(valLoader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            "epochs":epoch + 1,
            "Trainning Loss":avg_train_loss,
            "Valid Loss": avg_val_loss,
            "Valid Acc": avg_val_accuracy,
            "Training Time": validation_time
        }
    )


print("")
print("Training Complete")
print("Total Time took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))




output.shape